In [ ]:
print("Вариант:", sum([ord(x) for x in u'Седых']) % 3 + 1)

import numpy as np

Вариант: 3


In [ ]:
class Model_1:
    def __init__(self, a_min, a_max, b_min, b_max, p1, p2, p3, limit=False):
        self.a_min = a_min
        self.a_max = a_max
        self.b_min = b_min
        self.b_max = b_max
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.limit = limit

    def sample_a(self):
        return np.random.uniform(self.a_min, self.a_max)

    def sample_b(self):
        return np.random.uniform(self.b_min, self.b_max)

    def sample_c(self, a, b):
        c_a = np.random.binomial(a, self.p1)
        c_b = np.random.binomial(b, self.p2)
        c = c_a + c_b
        if self.limit:
            c = np.clip(c, 0, self.a_max + self.b_max)
        return c

    def sample_d(self, c):
        c = max(c, 1e-8)  # Минимальное значение для c, чтобы избежать ошибок
        din_sample = np.random.negative_binomial(c, self.p3)
        d = c + din_sample
        if self.limit:
            d = np.clip(d, 0, 2 * (self.a_max + self.b_max))
        return d

    def joint_probability(self):
        a = self.sample_a()
        b = self.sample_b()
        c = self.sample_c(a, b)
        d = self.sample_d(c)
        return a, b, c, d

    def generate_samples(self, n_samples):
        samples = [self.joint_probability() for _ in range(n_samples)]
        return pd.DataFrame(samples, columns=['a', 'b', 'c', 'd'])


class Model_2(Model_1):
    def sample_c(self, a, b):
        mean = a * self.p1 + b * self.p2
        c = np.random.poisson(mean)
        if self.limit:
            c = np.clip(c, 0, self.a_max + self.b_max)
        return c

    def sample_d(self, c):
        c = max(c, 1e-8)  # Минимальное значение для c, чтобы избежать ошибок
        bin_sample = np.random.binomial(c, self.p3)
        d = c + bin_sample
        if self.limit:
            d = np.clip(d, 0, 2 * (self.a_max + self.b_max))
        return d

In [ ]:
a_min = 75
a_max = 90
b_min = 500
b_max = 600
p1 = 0.1
p2 = 0.01
p3 = 0.3

model_1 = Model_1(a_min = a_min, a_max = a_max, b_min = b_min, b_max = b_max, p1 = p1, p2 = p2, p3 = p3, limit = True)
samples_model_1 = model_1.generate_samples(1000)
stats_model_1 = samples_model_1.describe().loc[['mean', 'std']]

In [2]:
print("Model 1: ")

# 1. Вычисление мат. ожиданий для a и b
mean_a = round(stats_model_1.loc['mean', 'a'])
mean_b = round(stats_model_1.loc['mean', 'b'])

# 2. Функция для вычисления c в зависимости от a и b
def calculate_c(a, b, p1, p2):
    c_a = np.random.binomial(a, p1, size=1000)  # Генерация выборки для c|a
    c_b = np.random.binomial(b, p2, size=1000)  # Генерация выборки для c|b
    return c_a, c_b

# 3. Поиск множества точек (p1, p2), при которых D[c|b] < D[c|a]
p1_values = np.linspace(0.01, 0.5, 50)
p2_values = np.linspace(0.01, 0.5, 50)

points_D_cb_less_D_ca = []
points_D_cb_greater_D_ca = []

for p1 in p1_values:
    for p2 in p2_values:
        c_a_samples, c_b_samples = calculate_c(mean_a, mean_b, p1, p2)

        D_ca = np.var(c_a_samples)  # Дисперсия c|a
        D_cb = np.var(c_b_samples)  # Дисперсия c|b

        if D_cb < D_ca:
            points_D_cb_less_D_ca.append((p1, p2))
        else:
            points_D_cb_greater_D_ca.append((p1, p2))

# 4. Преобразуем найденные точки в массивы для удобства визуализации
points_D_cb_less_D_ca = np.array(points_D_cb_less_D_ca)
points_D_cb_greater_D_ca = np.array(points_D_cb_greater_D_ca)

# 5. Построение графика множества точек (p1, p2)
plt.figure(figsize=(8, 6))

# Множество точек, где D[c|b] < D[c|a]
plt.scatter(points_D_cb_less_D_ca[:, 0], points_D_cb_less_D_ca[:, 1], color='green', label='D[c|b] < D[c|a]')

# Множество точек, где D[c|b] > D[c|a]
plt.scatter(points_D_cb_greater_D_ca[:, 0], points_D_cb_greater_D_ca[:, 1], color='red', label='D[c|b] > D[c|a]')

# Настройка графика
plt.title('Model 1 Множества точек (p1, p2)')
plt.xlabel('p1')
plt.ylabel('p2')
plt.legend()
plt.grid(True)
plt.show()

# 6. Проверка линейной разделимости множеств
from sklearn.svm import SVC

# Создадим набор данных для классификации
X = np.vstack((points_D_cb_less_D_ca, points_D_cb_greater_D_ca))
y = np.hstack((np.zeros(len(points_D_cb_less_D_ca)), np.ones(len(points_D_cb_greater_D_ca))))

# Обучим линейный классификатор
clf = SVC(kernel='linear')
clf.fit(X, y)

# Проверим точность разделения
print(f"Линейная разделимость: { clf.score(X, y) } - ", clf.score(X, y) > 0.95)






print("\n\n\nModel 2: ")
# 1. Вычисление мат. ожиданий для a и b
mean_a = round(stats_model_2.loc['mean', 'a'])
mean_b = round(stats_model_2.loc['mean', 'b'])

# 2. Функция для вычисления c в зависимости от a и b
def calculate_c(a, b, p1, p2):
    c_a = np.random.binomial(a, p1, size=1000)  # Генерация выборки для c|a
    c_b = np.random.binomial(b, p2, size=1000)  # Генерация выборки для c|b
    return c_a, c_b

# 3. Поиск множества точек (p1, p2), при которых D[c|b] < D[c|a]
p1_values = np.linspace(0.01, 0.5, 50)
p2_values = np.linspace(0.01, 0.5, 50)

points_D_cb_less_D_ca = []
points_D_cb_greater_D_ca = []

for p1 in p1_values:
    for p2 in p2_values:
        c_a_samples, c_b_samples = calculate_c(mean_a, mean_b, p1, p2)

        D_ca = np.var(c_a_samples)  # Дисперсия c|a
        D_cb = np.var(c_b_samples)  # Дисперсия c|b

        if D_cb < D_ca:
            points_D_cb_less_D_ca.append((p1, p2))
        else:
            points_D_cb_greater_D_ca.append((p1, p2))

# 4. Преобразуем найденные точки в массивы для удобства визуализации
points_D_cb_less_D_ca = np.array(points_D_cb_less_D_ca)
points_D_cb_greater_D_ca = np.array(points_D_cb_greater_D_ca)

# 5. Построение графика множества точек (p1, p2)
plt.figure(figsize=(8, 6))

# Множество точек, где D[c|b] < D[c|a]
plt.scatter(points_D_cb_less_D_ca[:, 0], points_D_cb_less_D_ca[:, 1], color='green', label='D[c|b] < D[c|a]')

# Множество точек, где D[c|b] > D[c|a]
plt.scatter(points_D_cb_greater_D_ca[:, 0], points_D_cb_greater_D_ca[:, 1], color='red', label='D[c|b] > D[c|a]')

# Настройка графика
plt.title('Model 2 Множества точек (p1, p2)')
plt.xlabel('p1')
plt.ylabel('p2')
plt.legend()
plt.grid(True)
plt.show()

# 6. Проверка линейной разделимости множеств
from sklearn.svm import SVC

# Создадим набор данных для классификации
X = np.vstack((points_D_cb_less_D_ca, points_D_cb_greater_D_ca))
y = np.hstack((np.zeros(len(points_D_cb_less_D_ca)), np.ones(len(points_D_cb_greater_D_ca))))

# Обучим линейный классификатор
clf = SVC(kernel='linear')
clf.fit(X, y)

# Проверим точность разделения
print(f"Линейная разделимость: { clf.score(X, y) } - ", clf.score(X, y) > 0.95)

Model 1: 


NameError: name 'stats_model_1' is not defined